In [1]:
# Resources below
# https://medium.com/@evergreenllc2020/semantic-search-engine-with-s-abbfb3cd9377
# https://github.com/evergreenllc2020/nlp/blob/master/semantic%20search%20%20with%20BERT/Semantic%20Search.ipynb

In [2]:
from sentence_transformers import SentenceTransformer
import scipy
import os
import pandas as pd

C:\Users\leefa_1yhwpme\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\leefa_1yhwpme\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\leefa_1yhwpme\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\leefa_1yhwpme\anaconda3\lib\site-packages\tensorflow\python\framework\dty

In [3]:
# Load the BERT model. Various models trained on Natural Language Inference (NLI) https://github.com/UKPLab/sentence-transformers/blob/master/docs/pretrained-models/nli-models.md and 
# Semantic Textual Similarity are available https://github.com/UKPLab/sentence-transformers/blob/master/docs/pretrained-models/sts-models.md

# takes 1 minute to run

model = SentenceTransformer('bert-base-nli-mean-tokens')

In [4]:
cd Desktop

C:\Users\leefa_1yhwpme\Desktop


In [5]:
cd Stemaway ML Workspace

C:\Users\leefa_1yhwpme\Desktop\Stemaway ML Workspace


In [6]:
df = pd.read_csv('ML Team 6 Data Processing - bagofwords.csv')
df.info()
df.head()
df = df.drop_duplicates(subset=['Topic'], keep='first').reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155182 entries, 0 to 155181
Data columns (total 2 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   Topic         155182 non-null  object
 1   Bag of Words  155182 non-null  object
dtypes: object(2)
memory usage: 2.4+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121884 entries, 0 to 121883
Data columns (total 2 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   Topic         121884 non-null  object
 1   Bag of Words  121884 non-null  object
dtypes: object(2)
memory usage: 1.9+ MB


In [7]:
# Takes 8 minutes to run

df = df.head(20000)
df.head(20)

sentences = df['Bag of Words'].values.tolist()

sentence_embeddings = model.encode(sentences)

print('Sample BERT embedding vector - length', len(sentence_embeddings[0]))

print('Sample BERT embedding vector - note includes negative values', sentence_embeddings[0])

Sample BERT embedding vector - length 768
Sample BERT embedding vector - note includes negative values [-2.49245167e-01  5.95455408e-01  4.76517886e-01 -4.12851602e-01
  4.24785167e-01 -2.79355645e-01  5.24952948e-01 -4.71364737e-01
  4.16098870e-02 -7.61938751e-01 -4.49587941e-01  4.82899249e-01
 -5.96702993e-01 -1.05041536e-02 -5.38355887e-01  2.08145663e-01
 -1.09632504e+00  2.20024914e-01 -5.31691611e-02  1.27221122e-01
  1.64966926e-01 -7.90561616e-01 -4.58885491e-01 -2.32428536e-02
  1.49314153e+00  7.92784765e-02 -1.49959326e-03 -1.85748726e-01
 -1.65816402e+00  2.71535128e-01 -9.85046566e-01 -5.15246630e-01
  3.48460734e-01 -9.13198054e-01 -3.81268531e-01  1.06575167e+00
 -4.29645866e-01  3.18409642e-03  6.16952956e-01  5.11002615e-02
 -1.82456702e-01 -5.65176487e-01  3.90103042e-01  3.33058238e-01
 -9.87904966e-01  3.66173200e-02 -6.82186723e-01  8.68938863e-01
  5.94652072e-02 -2.84962654e-01 -7.56599605e-01  3.60609680e-01
  4.65747088e-01  2.81810537e-02 -4.38328177e-01 -1.

In [8]:
# code adapted from https://github.com/UKPLab/sentence-transformers/blob/master/examples/application_semantic_search.py

#query = 'bad weather' #@param {type: 'string'}
#query = 'employee stop working' #@param {type: 'string'}
#query = "moderate lift in economy"
#query = 'global warming impact'
query = 'How to save a file downloaded from requests to another directory?'

# Test the following topics below:
# Joining arrays within group by clause
# How to save a file downloaded from requests to another directory?
# All keywords for the 'Restrictons of Python compared to Ruby: lambda's' topic = cosine score of 1:
# python ruby lambda restriction compared restrictons would like expression wikivs going quote know good cannot contain statement restricted page

# works for any given query

queries = [query]
query_embeddings = model.encode(queries)

# Find the closest 3 sentences of the corpus for each query sentence based on cosine similarity
number_top_matches = 10 #@param {type: "number"}

print("Semantic Search Results")

for query, query_embedding in zip(queries, query_embeddings):
    distances = scipy.spatial.distance.cdist([query_embedding], sentence_embeddings, "cosine")[0]

    results = zip(range(len(distances)), distances)
    results = sorted(results, key=lambda x: x[1])

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop 10 most similar sentences in corpus:")

    for idx, distance in results[0:number_top_matches]:
        if df['Topic'][idx] == query:
            print('*Duplicate Topic*   ',df['Topic'][idx], "(Cosine Score: %.4f)" % (1-distance))
        else:
            print(df['Topic'][idx], "(Cosine Score: %.4f)" % (1-distance))

Semantic Search Results




Query: How to save a file downloaded from requests to another directory?

Top 10 most similar sentences in corpus:
*Duplicate Topic*    How to save a file downloaded from requests to another directory? (Cosine Score: 0.7845)
How to copy all files from a folder (including sub-folder) while not copying the folder structure in python (Cosine Score: 0.7528)
Saving and loading a Numpy Matrix in python (Cosine Score: 0.7299)
Where to find/create .git/config file for local configuration for project? [duplicate] (Cosine Score: 0.7293)
git and C++ workflow, how to handle object and archive files? (Cosine Score: 0.7288)
how to delete json object using python? (Cosine Score: 0.7266)
docker add cache when git checkout same file (Cosine Score: 0.7261)
Repository with submodules after rewriting history of submodule (Cosine Score: 0.7260)
Delete Image Files From Server (Cosine Score: 0.7249)
how can I export mysql database using ssh? (Cosine Score: 0.7248)
